<h1> DattoRMM - ConnectWise Products Installed </h1>

# Import Modules, Call API, Create DataFrame
## Initialize Variables for API
### Import Modules

In [ ]:
import re

import pandas as pd
import numpy as np
import datetime as dt
[REDACTED]/.pyplot as plt
import seaborn as sns
import json

### Get Current Time and Set Export Folder

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

### Pull Environment Variables

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# import and assign secrets from env.ini
base_uri = config['dattormm']['base_uri']
api_key = config['dattormm']['api_key']
api_secret = config['dattormm']['api_secret']

## Create Monitors DataFrame
### Get Oauth Token

In [ ]:
# call token api url
token_uri = f'{base_uri}/auth/oauth/token'

# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = api_key
data['password'] = api_secret

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

### API GET Open Monitors DataFrame

In [ ]:
# request content response
request_url = f'{base_uri}/api/v2/account/alerts/open'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)
print(c_dict['pageDetails']['nextPageUrl'])

# iterate and combine remaining pages
df_alerts = pd.DataFrame(c_dict['alerts'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    print(c_dict['pageDetails']['nextPageUrl'])
    df_current_page = pd.DataFrame(c_dict['alerts'])
    df_alerts = pd.concat([df_alerts, df_current_page], ignore_index=False)

## Create Columns from Column Dictionaries
alertMonitorInfo | alertContext | alertSourceInfo | responseActions

### alertMonitorInfo

In [ ]:
# sendsEmails
def sendsEmails(alertMonitorInfo):
    return alertMonitorInfo['sendsEmails']


df_alerts['sendsEmails'] = df_alerts['alertMonitorInfo'].apply(sendsEmails)


# sendsEmails
def createsTicket(alertMonitorInfo):
    return alertMonitorInfo['createsTicket']


df_alerts['createsTicket'] = df_alerts['alertMonitorInfo'].apply(createsTicket)

df_alerts.drop('alertMonitorInfo', axis=1, inplace=True)


### alertContext

In [ ]:
# @class
def alert_class(alertContext):
    #print(alertContext.keys())
    if alertContext['@class'] == 'comp_script_ctx':
        return (alertContext['@class'])


df_alerts['alertClass'] = df_alerts['alertContext'].apply(alert_class)

# status
def status(alertContext):
    if alertContext['@class'] == 'comp_script_ctx':
        return (alertContext['samples'])
    #print(alertContext)

df_alerts['alertClass'] = df_alerts['alertContext'].apply(alert_class)
df_alerts['status'] = df_alerts['alertContext'].apply(status)


df_alerts.drop('alertContext', axis=1, inplace=True)

In [ ]:
status_prog = re.compile(r'\{\'([^\'\"]*)[\s\"\':]+([\s\w\d\.\-]*)')
def status_type(status):
    if status == None:
        return None
    else:
        status_type = status_prog.findall(str(status))
        return (status_type[0][0])

def status_message(status):
    if status == None:
        return None
    else:
        status_type = status_prog.findall(str(status))
        return (status_type[0][1])
df_alerts['statusType'] = df_alerts['status'].apply(status_type)
df_alerts['statusMessage'] = df_alerts['status'].apply(status_message)
df_alerts.drop('status', axis=1, inplace=True)

### diagnostics

In [ ]:
diagnostics_prog = re.compile(r'([\w\d\/\\\:\#\=\.\-\s]+)*')
def diagnostics_split(diagnostics):
    if status == None:
        return None
    else:
        diagnostic_split = diagnostics_prog.findall(str(diagnostics))
        return ''.join(diagnostic_split)

df_alerts['diagnosticsMessage'] = df_alerts['diagnostics'].apply(diagnostics_split)
df_alerts.drop('diagnostics', axis=1 , inplace=True)

### alertSourceInfo

In [ ]:
def deviceUid(alertSourceInfo):
    if alertSourceInfo == None:
        return None
    else:
        return alertSourceInfo['deviceUid']


def deviceName(alertSourceInfo):
    if alertSourceInfo == None:
        return None
    else:
        return alertSourceInfo['deviceName']


def siteUid(alertSourceInfo):
    if alertSourceInfo == None:
        return None
    else:
        return alertSourceInfo['siteUid']


def siteName(alertSourceInfo):
    if alertSourceInfo == None:
        return None
    else:
        return alertSourceInfo['siteName']

In [ ]:
df_alerts['deviceUid'] = df_alerts['alertSourceInfo'].apply(deviceUid)
df_alerts['deviceName'] = df_alerts['alertSourceInfo'].apply(deviceName)
df_alerts['siteUid'] = df_alerts['alertSourceInfo'].apply(siteUid)
df_alerts['siteName'] = df_alerts['alertSourceInfo'].apply(siteName)
df_alerts.drop('alertSourceInfo',axis=1, inplace=True)

### responseActions

In [ ]:
def action_time(responseActions):
    if responseActions == None:
        return None
    else:
        return responseActions['actionTime']


def actionType(responseActions):
    if responseActions == None:
        return None
    else:
        return responseActions['actionType']


def description(responseActions):
    if responseActions == None:
        return None
    else:
        return responseActions['description']


def actionReference(responseActions):
    if responseActions == None:
        return None
    else:
        return responseActions['actionReference']


def action_time(responseActions):
    if responseActions == None:
        return None
    else:
        return responseActions['actionReferenceInt']

In [ ]:
for row in df_alerts['responseActions']:
    if row == None:
        continue
    else:
        print(len(row))